# GPU Setting

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
USE_GPU = True

if USE_GPU and tf.config.list_physical_devices('GPU'):
    device = 'GPU'
    print("Using GPU")
else:
    device = 'CPU'
    print("Using CPU")

2024-11-22 15:49:04.862659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 15:49:04.961693: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Num GPUs Available:  1
Using GPU


2024-11-22 15:49:05.919411: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-22 15:49:05.938961: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-22 15:49:05.939113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Loading Dataset

In [2]:
import tarfile
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import random
import torch

# Path to the downloaded tgz file
tgz_path = "/home/asko/Documents/workspace/Fall-24/682/project/dataset/imagenette2.tgz"
extract_path = "./imagenette"  # Target folder for extraction

# Extract the file
with tarfile.open(tgz_path, "r:gz") as tar:
    tar.extractall(path=extract_path)
print("Extraction completed.")


transform = transforms.Compose([
    transforms.Resize((224,224)),  # Resize images to a size suitable for VGG16
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per VGG16
])
extract_path_train = "./imagenette/imagenette2/train"
train_dataset = datasets.ImageFolder(
    root=extract_path_train,  # Imagenette URL
    transform=transform
)
extract_path_val = "./imagenette/imagenette2/val"
val_dataset = datasets.ImageFolder(
    root=extract_path_val,  # Imagenette URL
    transform=transform
)

batch_size = 8
sampler_train = sampler.SubsetRandomSampler(range(len(train_dataset)))
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler_train)

batch_size = 8
sampler_val = sampler.SubsetRandomSampler(range(1000))
val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler_val)

# batch_size = 32
# sampler_test = sampler.SubsetRandomSampler(range(1000, len(val_dataset)))
# test_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler_test)


/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extraction completed.


In [3]:
import tensorflow as tf6559
from CustomArchitecture.Custom_Lite_TACNN import Custom_Lite_TACNN

alpha = 0.5
temperature = 3

# Load the VGG16 Teacher model
import dill
with open("custom_lite_ta1CNN.pkl", "rb") as f:
    ta1_model = dill.load(f)

# Load the VGG16 model
ta2_model = Custom_Lite_TACNN(alpha=alpha, temperature=temperature, num_of_classes=10)

device = 'cuda' if len(tf.config.list_physical_devices('GPU'))!=0 else 'cpu'
ta1_model.to(device)
ta2_model.to(device)
print(device)


cuda


In [ ]:
# teacher_outputs = []
# ta1_model.eval()
# for batch_idx, (images, _) in enumerate(train_loader):
#     images = images.to(device, dtype=torch.float32)
#     torch.save(ta1_model(images), f"outputs/Custom_TA1/ta1_outputs_{batch_idx}.pt")

# del ta1_model


In [6]:
def check_accuracy_part34(loader, model):
    print('Checking accuracy on validation set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))


In [9]:
import torch.optim as optim
import torch
import torch.nn.functional as F

optimizer = optim.Adam(ta2_model.parameters(), lr=0.01)
# optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-5, momentum=0.98, nesterov=False)
torch.set_grad_enabled(True)

num_epochs = 50
# batch_size = 16  
model_weights = None
print_every = 100


for epoch in range(num_epochs): 
    for batch_idx, (images, labels) in enumerate(train_loader):
        ta2_model.train()           
        
        # ta1_model.eval()
        
        images = images.to(device, dtype=torch.float32)
        labels = labels.to(device, dtype=torch.long)
        
        ta1_output = torch.load(f"outputs/Custom_TA1/ta1_outputs_{batch_idx}.pt")
        ta2_output = ta2_model(images)
        loss = ta2_model.risk(Y=labels, teacher_preds=ta1_output, output=ta2_output)
        # print(ta1_model.l_feat)
        del ta1_output
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % print_every == 0:
            print('Iteration %d, loss = %.4f' % (batch_idx, loss.item()))
            check_accuracy_part34(val_loader, ta2_model)
            print()
        
        torch.cuda.empty_cache()
        del ta2_output, images, labels, loss
        
    print(f"Epoch {epoch} complete")

/home/asko/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/torch/nn/functional.py:2916: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Iteration 0, loss = 1.9004
Checking accuracy on validation set
Got 0 / 1000 correct (0.00)

Iteration 100, loss = 4.4450
Checking accuracy on validation set
Got 77 / 1000 correct (7.70)

Iteration 200, loss = 1.6809
Checking accuracy on validation set
Got 483 / 1000 correct (48.30)

Iteration 300, loss = 1.9977
Checking accuracy on validation set
Got 572 / 1000 correct (57.20)

Iteration 400, loss = 1.8314
Checking accuracy on validation set
Got 503 / 1000 correct (50.30)

Iteration 500, loss = 1.8931
Checking accuracy on validation set
Got 383 / 1000 correct (38.30)

Iteration 600, loss = 1.9888
Checking accuracy on validation set
Got 374 / 1000 correct (37.40)

Iteration 700, loss = 1.4708
Checking accuracy on validation set
Got 467 / 1000 correct (46.70)

Iteration 800, loss = 1.7015
Checking accuracy on validation set
Got 563 / 1000 correct (56.30)

Iteration 900, loss = 1.7545
Checking accuracy on validation set
Got 429 / 1000 correct (42.90)

Iteration 1000, loss = 1.6608
Checkin

KeyboardInterrupt: 

In [10]:
import dill
with open("custom_lite_ta2CNN.pkl", "wb") as f:
    dill.dump(ta2_model, f)

In [ ]:

batch_size = 16
sampler_test = sampler.SubsetRandomSampler(range(1000, len(val_dataset)))
test_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler_test)

check_accuracy_part34(test_loader, ta2_model)

Checking accuracy on validation set
Got 2037 / 2925 correct (69.64)


: 

In [ ]:
del test_loader, ta2_model